# Advanced Support Agent PoC with LangGraph

This notebook demonstrates a Proof of Concept (PoC) for a robust support agent using the LangGraph library. The agent simulates a support bot capable of performing various tasks such as log analysis, alerting, ticket management, engineer escalation, and knowledge base querying.

## Objective
- Demonstrate the flexibility and adaptability of LangGraph.
- Simulate a support bot with dynamic decision-making and error recovery.
- Use mock implementations for external integrations to focus on LangGraph's capabilities.

## 1. Setup and Dependencies

In [901]:
import os
import random
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, END

# Mock LLM for generating responses
def mock_llm(prompt):
    return f"Mock response for: {prompt}"

print("✅ Dependencies loaded successfully")

✅ Dependencies loaded successfully


## 2. State Definition

Define the state structure for tracking the support workflow progress.

In [902]:
class SupportState(TypedDict):
    user_query: str
    issue: str
    severity: str
    identifiers: dict
    logs: str
    analysis: str
    decision: str
    alert: str
    ticket_id: str
    escalation: str
    kb_response: str
    response: str
    error_message: str
    attempt_count: int

print("✅ State definition complete")

✅ State definition complete


## 3. Mock Tools

These tools simulate external integrations such as log fetchers, ticketing systems, and alerting mechanisms.

In [ ]:
# Mock log fetcher with error recovery
def fetch_logs(state: SupportState):
    """Simulate fetching logs with random failures for testing error recovery"""
    attempt = state.get('attempt_count', 1)
    print(f"🔄 Attempting to fetch logs (attempt {attempt})...")
    
    if random.random() < 0.5:  # 50% success rate
        # Generate varied log content based on issue type and severity
        issue_type = state.get('issue', 'general')
        initial_severity = state.get('severity', 'low')
        order_id = state.get('identifiers', {}).get('order_id', 'unknown')
        
        # Create realistic log scenarios
        if issue_type == "payment" and initial_severity == "high":
            # 50% chance of actual error for payment issues
            if random.random() < 0.5:
                state["logs"] = f"Log data for {order_id}: ERROR - Payment processing failed at timestamp 2024-01-15T10:30:00Z. Gateway timeout after 30s."
            else:
                state["logs"] = f"Log data for {order_id}: INFO - Payment processing completed successfully. Transaction ID: TXN-789. Duration: 2.3s."
        elif issue_type == "login":
            # Login issues might have authentication errors or just warnings
            if random.random() < 0.3:
                state["logs"] = f"Log data: ERROR - Authentication failed for user. Invalid credentials provided."
            else:
                state["logs"] = f"Log data: WARN - Multiple login attempts detected. Rate limiting applied. User authenticated successfully on retry."
        else:
            # General issues usually have info or warning logs
            log_types = [
                f"Log data: INFO - Request processed successfully. Response time: 150ms.",
                f"Log data: WARN - Slow response detected. Query took 5.2s to complete.",
                f"Log data: INFO - Cache miss for user session. Data retrieved from database.",
                f"Log data: DEBUG - API endpoint /health returned 200 OK."
            ]
            state["logs"] = random.choice(log_types)
            
        print("✅ Logs fetched successfully")
        print(f"   Content preview: {state['logs'][:80]}...")
        return state
    else:
        # Check if we've hit max attempts BEFORE incrementing
        if attempt >= 3:
            state["error_message"] = "Failed to fetch logs after 3 attempts"
            print("❌ Max retry attempts reached for log fetching")
            return state
        else:
            # Increment attempt count for next retry
            state["attempt_count"] = attempt + 1
            # Set a temporary error message to signal retry needed
            state["error_message"] = "retry_needed"
            print(f"   Will retry (attempt {attempt + 1})")
        return state

# Mock log analyzer with intelligent analysis
def analyze_logs(state: SupportState):
    """Analyze logs and determine severity and next actions"""
    logs = state.get("logs", "")
    current_severity = state.get("severity", "low")
    
    print(f"🔍 Analyzing logs with initial severity: {current_severity}")
    
    # Analyze log content to determine actual severity
    if "ERROR" in logs:
        if "Payment" in logs or "Authentication failed" in logs:
            state["analysis"] = "Critical system failure detected"
            state["severity"] = "critical"
            print("🔍 Analysis: Critical error found - upgrading severity")
        else:
            state["analysis"] = "Error condition detected in logs"
            state["severity"] = "high"
            print("🔍 Analysis: Error found - setting high severity")
    elif "WARN" in logs:
        state["analysis"] = "Warning condition detected - monitoring required"
        # Keep existing severity or set to medium if it was low
        if current_severity == "low":
            state["severity"] = "medium"
        print("🔍 Analysis: Warning found - moderate severity")
    else:
        state["analysis"] = "No critical issues found in logs - normal operation"
        state["severity"] = "low"
        print("🔍 Analysis: Normal logs - keeping low severity")
    
    return state

# Mock alerting system
def trigger_alert(state: SupportState):
    """Trigger alerts for critical issues"""
    severity = state.get("severity", "low")
    analysis = state.get("analysis", "")
    
    alert_message = f"CRITICAL ALERT: {analysis} - Severity: {severity}"
    print(f"🚨 {alert_message}")
    state["alert"] = alert_message
    state["decision"] = "alert_triggered"
    return state

# Mock ticketing system
def create_ticket(state: SupportState):
    """Create support tickets for tracking issues"""
    ticket_id = f"TICKET-{random.randint(1000, 9999)}"
    issue_summary = state.get("analysis", "Support issue")
    
    print(f"🎫 Ticket created: {ticket_id}")
    print(f"   Issue: {issue_summary}")
    print(f"   Severity: {state.get('severity', 'unknown')}")
    
    state["ticket_id"] = ticket_id
    state["decision"] = "ticket_created"
    return state

# Mock engineer notification
def notify_engineer(state: SupportState):
    """Notify engineers for escalation"""
    reason = state.get("error_message", "Log fetch failure") or "Critical issue escalation"
    print(f"📢 Engineer notified for escalation: {reason}")
    state["escalation"] = f"Engineer notified: {reason}"
    state["decision"] = "escalated"
    return state

# Mock knowledge base query
def query_knowledge_base(state: SupportState):
    """Search knowledge base for relevant solutions"""
    issue = state.get("issue", "general")
    
    # Simulate KB responses based on issue type
    kb_responses = {
        "payment": "KB Article: Payment Processing Troubleshooting - Check gateway status and retry mechanisms",
        "login": "KB Article: Authentication Issues - Verify user credentials and session management",
        "general": "KB Article: General Troubleshooting - Standard diagnostic procedures"
    }
    
    response = kb_responses.get(issue, kb_responses["general"])
    print(f"📚 Knowledge base queried: {response[:50]}...")
    state["kb_response"] = response
    return state

print("✅ Mock tools defined")

✅ Mock tools defined


## 4. LangGraph Nodes

Define the workflow nodes with enhanced logic and error handling.

In [904]:
def extract_details(state: SupportState):
    """Extract issue details from user query"""
    query = state.get("user_query", "Payment processing issue with order")
    
    # Simulate intelligent extraction
    if "payment" in query.lower():
        state["issue"] = "payment"
        state["severity"] = "high"
        state["identifiers"] = {"order_id": "ORD-12345", "user_id": "USR-789"}
    elif "login" in query.lower():
        state["issue"] = "login" 
        state["severity"] = "medium"
        state["identifiers"] = {"user_id": "USR-456", "session_id": "SES-123"}
    else:
        state["issue"] = "general"
        state["severity"] = "low"
        state["identifiers"] = {"request_id": "REQ-999"}
    
    print(f"✅ Details extracted - Issue: {state['issue']}, Severity: {state['severity']}")
    state["attempt_count"] = 1
    return state

def retry_fetch_logs(state: SupportState):
    """Retry log fetching with exponential backoff simulation"""
    print("🔄 Retrying log fetch with enhanced parameters...")
    return fetch_logs(state)

def decision_router(state: SupportState):
    """Smart routing based on analysis and state"""
    analysis = state.get("analysis", "")
    severity = state.get("severity", "low")
    logs_available = bool(state.get("logs"))
    error_occurred = bool(state.get("error_message"))
    
    print(f"🧠 Decision router - Severity: {severity}, Logs: {logs_available}, Error: {error_occurred}")
    
    # Route based on conditions
    if error_occurred and not logs_available:
        return "escalate"
    elif severity == "critical":
        return "alert"
    elif severity in ["high", "medium"]:
        return "ticket"
    else:
        return "knowledge_base"

def final_response(state: SupportState):
    """Generate final response summarizing actions taken"""
    decision = state.get("decision", "unknown")
    ticket_id = state.get("ticket_id", "")
    alert = state.get("alert", "")
    escalation = state.get("escalation", "")
    kb_response = state.get("kb_response", "")
    
    response_parts = ["Support workflow completed."]
    
    if alert:
        response_parts.append(f"🚨 Alert triggered: {alert}")
    if ticket_id:
        response_parts.append(f"🎫 Ticket created: {ticket_id}")
    if escalation:
        response_parts.append(f"📢 Escalated: {escalation}")
    if kb_response:
        response_parts.append(f"📚 KB Reference: {kb_response}")
    
    final_msg = " | ".join(response_parts)
    print(f"🎯 {final_msg}")
    state["response"] = final_msg
    return state

print("✅ LangGraph nodes defined")

✅ LangGraph nodes defined


## 5. Advanced LangGraph Workflow

Build a sophisticated workflow with dynamic routing, error recovery, and multi-path decision making.

In [905]:
def create_advanced_support_agent():
    """Create the advanced support agent with comprehensive workflow"""
    
    workflow = StateGraph(SupportState)
    
    # Add all nodes
    workflow.add_node("extract_details", extract_details)
    workflow.add_node("fetch_logs", fetch_logs)
    workflow.add_node("retry_fetch_logs", retry_fetch_logs)
    workflow.add_node("analyze_logs", analyze_logs)
    workflow.add_node("trigger_alert", trigger_alert)
    workflow.add_node("create_ticket", create_ticket)
    workflow.add_node("notify_engineer", notify_engineer)
    workflow.add_node("query_knowledge_base", query_knowledge_base)
    workflow.add_node("final_response", final_response)
    
    # Set entry point
    workflow.set_entry_point("extract_details")
    
    # Basic flow: extract -> fetch logs
    workflow.add_edge("extract_details", "fetch_logs")
    
    # FIXED: Conditional routing from fetch_logs
    def route_after_fetch(state):
        error_message = state.get("error_message", "")
        logs_available = bool(state.get("logs"))
        current_attempt = state.get("attempt_count", 1)
        
        print(f"🔀 Routing decision: attempt={current_attempt}, error_msg='{error_message}', logs={logs_available}")
        
        # Check conditions in the right order
        if logs_available:
            # Successfully got logs - proceed to analysis
            return "analyze"
        elif error_message == "retry_needed" and current_attempt <= 3:
            # Need to retry and haven't exceeded max attempts
            return "retry"
        elif "Failed to fetch logs after" in error_message:
            # Exhausted all retries - escalate
            return "escalate"
        else:
            # Any other error condition - escalate
            return "escalate"
    
    workflow.add_conditional_edges(
        "fetch_logs",
        route_after_fetch,
        {
            "retry": "retry_fetch_logs",
            "analyze": "analyze_logs", 
            "escalate": "notify_engineer"
        }
    )
    
    # Retry can loop back or escalate
    workflow.add_conditional_edges(
        "retry_fetch_logs", 
        route_after_fetch,
        {
            "retry": "retry_fetch_logs",
            "analyze": "analyze_logs",
            "escalate": "notify_engineer"
        }
    )
    
    # Smart routing after analysis
    workflow.add_conditional_edges(
        "analyze_logs",
        decision_router,
        {
            "alert": "trigger_alert",
            "ticket": "create_ticket", 
            "escalate": "notify_engineer",
            "knowledge_base": "query_knowledge_base"
        }
    )
    
    # All paths lead to final response
    workflow.add_edge("trigger_alert", "final_response")
    workflow.add_edge("create_ticket", "final_response") 
    workflow.add_edge("notify_engineer", "final_response")
    workflow.add_edge("query_knowledge_base", "final_response")
    workflow.add_edge("final_response", END)
    
    return workflow.compile()

# Create the advanced agent
advanced_support_agent = create_advanced_support_agent()
print("✅ Advanced support agent workflow created")

✅ Advanced support agent workflow created


## 6. Testing Scenarios

Test the agent with various scenarios to demonstrate adaptability and robustness.

In [906]:
def test_scenario(scenario_name: str, user_query: str):
    """Test a specific scenario and display results"""
    print("="*80)
    print(f"🧪 TESTING SCENARIO: {scenario_name}")
    print("="*80)
    print(f"User Query: '{user_query}'")
    print("-"*40)
    
    initial_state = SupportState(
        user_query=user_query,
        issue="",
        severity="",
        identifiers={},
        logs="",
        analysis="",
        decision="",
        alert="",
        ticket_id="",
        escalation="",
        kb_response="",
        response="",
        error_message="",
        attempt_count=1
    )
    
    try:
        result = advanced_support_agent.invoke(initial_state)
        
        print("\n📊 SCENARIO RESULTS:")
        print("-"*40)
        print(f"✅ Final Response: {result.get('response', 'No response')}")
        print(f"🔍 Analysis: {result.get('analysis', 'No analysis')}")
        print(f"⚡ Decision: {result.get('decision', 'No decision')}")
        print(f"🔄 Attempts: {result.get('attempt_count', 1)}")
        
        if result.get('error_message'):
            print(f"⚠️ Errors: {result['error_message']}")
            
        return result
        
    except Exception as e:
        print(f"❌ Scenario failed: {str(e)}")
        return None

print("✅ Testing framework ready")

✅ Testing framework ready


In [907]:
# Test Scenario 1: Critical Payment Issue
result1 = test_scenario(
    "Critical Payment Processing", 
    "Customer reporting payment processing failure for order ORD-12345"
)

🧪 TESTING SCENARIO: Critical Payment Processing
User Query: 'Customer reporting payment processing failure for order ORD-12345'
----------------------------------------
✅ Details extracted - Issue: payment, Severity: high
🔄 Attempting to fetch logs (attempt 1)...
   Will retry (attempt 2)
🔀 Routing decision: attempt=2, error_msg='retry_needed', logs=False
🔄 Retrying log fetch with enhanced parameters...
🔄 Attempting to fetch logs (attempt 2)...
   Will retry (attempt 3)
🔀 Routing decision: attempt=3, error_msg='retry_needed', logs=False
🔄 Retrying log fetch with enhanced parameters...
🔄 Attempting to fetch logs (attempt 3)...
❌ Max retry attempts reached for log fetching
🔀 Routing decision: attempt=3, error_msg='Failed to fetch logs after 3 attempts', logs=False
📢 Engineer notified for escalation: Failed to fetch logs after 3 attempts
🎯 Support workflow completed. | 📢 Escalated: Engineer notified: Failed to fetch logs after 3 attempts

📊 SCENARIO RESULTS:
------------------------------

In [908]:
# Test Scenario 2: Login Authentication Issue  
result2 = test_scenario(
    "Login Authentication Problem",
    "User unable to login to their account - authentication failing"
)

🧪 TESTING SCENARIO: Login Authentication Problem
User Query: 'User unable to login to their account - authentication failing'
----------------------------------------
✅ Details extracted - Issue: login, Severity: medium
🔄 Attempting to fetch logs (attempt 1)...
   Will retry (attempt 2)
🔀 Routing decision: attempt=2, error_msg='retry_needed', logs=False
🔄 Retrying log fetch with enhanced parameters...
🔄 Attempting to fetch logs (attempt 2)...
✅ Logs fetched successfully
   Content preview: Log data: WARN - Multiple login attempts detected. Rate limiting applied. User a...
🔀 Routing decision: attempt=2, error_msg='retry_needed', logs=True
🔍 Analyzing logs with initial severity: medium
🔍 Analysis: Warning found - moderate severity
🧠 Decision router - Severity: medium, Logs: True, Error: True
🎫 Ticket created: TICKET-5534
   Issue: Warning condition detected - monitoring required
   Severity: medium
🎯 Support workflow completed. | 🎫 Ticket created: TICKET-5534

📊 SCENARIO RESULTS:
-------

In [909]:
# Test Scenario 3: General Support Query
result3 = test_scenario(
    "General Support Issue",
    "User experiencing slow loading times on the website"
)

🧪 TESTING SCENARIO: General Support Issue
User Query: 'User experiencing slow loading times on the website'
----------------------------------------
✅ Details extracted - Issue: general, Severity: low
🔄 Attempting to fetch logs (attempt 1)...
   Will retry (attempt 2)
🔀 Routing decision: attempt=2, error_msg='retry_needed', logs=False
🔄 Retrying log fetch with enhanced parameters...
🔄 Attempting to fetch logs (attempt 2)...
   Will retry (attempt 3)
🔀 Routing decision: attempt=3, error_msg='retry_needed', logs=False
🔄 Retrying log fetch with enhanced parameters...
🔄 Attempting to fetch logs (attempt 3)...
❌ Max retry attempts reached for log fetching
🔀 Routing decision: attempt=3, error_msg='Failed to fetch logs after 3 attempts', logs=False
📢 Engineer notified for escalation: Failed to fetch logs after 3 attempts
🎯 Support workflow completed. | 📢 Escalated: Engineer notified: Failed to fetch logs after 3 attempts

📊 SCENARIO RESULTS:
----------------------------------------
✅ Final Re

## 7. Performance Analysis

Compare the agent's behavior across different scenarios and analyze decision patterns.

In [910]:
def analyze_agent_performance():
    """Analyze the agent's performance across multiple test runs"""
    print("="*80)
    print("📈 AGENT PERFORMANCE ANALYSIS")
    print("="*80)
    
    test_cases = [
        ("Payment Issue", "Payment processing error for order ORD-456"),
        ("Login Problem", "Cannot authenticate user login credentials"), 
        ("General Query", "Website performance issues reported"),
        ("Critical System", "System-wide outage affecting all users"),
        ("Data Issue", "Data synchronization problems in user profiles")
    ]
    
    results = []
    for case_name, query in test_cases:
        print(f"\n🔄 Running: {case_name}")
        
        state = SupportState(
            user_query=query, issue="", severity="", identifiers={},
            logs="", analysis="", decision="", alert="", ticket_id="",
            escalation="", kb_response="", response="", error_message="", attempt_count=1
        )
        
        try:
            result = advanced_support_agent.invoke(state)
            results.append({
                'case': case_name,
                'success': True,
                'decision': result.get('decision', 'unknown'),
                'severity': result.get('severity', 'unknown'),
                'attempts': result.get('attempt_count', 1)
            })
        except Exception as e:
            results.append({
                'case': case_name, 
                'success': False,
                'error': str(e),
                'attempts': 1
            })
    
    # Analysis summary
    print("\n📊 PERFORMANCE SUMMARY:")
    print("-"*40)
    success_rate = sum(1 for r in results if r['success']) / len(results) * 100
    print(f"Success Rate: {success_rate:.1f}%")
    
    decision_types = {}
    total_attempts = 0
    
    for result in results:
        if result['success']:
            decision = result['decision']
            decision_types[decision] = decision_types.get(decision, 0) + 1
            total_attempts += result['attempts']
    
    print(f"Average Attempts: {total_attempts/len([r for r in results if r['success']]):.1f}")
    print(f"Decision Distribution: {decision_types}")
    
    return results

# Run performance analysis
performance_results = analyze_agent_performance()

📈 AGENT PERFORMANCE ANALYSIS

🔄 Running: Payment Issue
✅ Details extracted - Issue: payment, Severity: high
🔄 Attempting to fetch logs (attempt 1)...
   Will retry (attempt 2)
🔀 Routing decision: attempt=2, error_msg='retry_needed', logs=False
🔄 Retrying log fetch with enhanced parameters...
🔄 Attempting to fetch logs (attempt 2)...
✅ Logs fetched successfully
   Content preview: Log data for ORD-12345: INFO - Payment processing completed successfully. Transa...
🔀 Routing decision: attempt=2, error_msg='retry_needed', logs=True
🔍 Analyzing logs with initial severity: high
🔍 Analysis: Normal logs - keeping low severity
🧠 Decision router - Severity: low, Logs: True, Error: True
📚 Knowledge base queried: KB Article: Payment Processing Troubleshooting - C...
🎯 Support workflow completed. | 📚 KB Reference: KB Article: Payment Processing Troubleshooting - Check gateway status and retry mechanisms

🔄 Running: Login Problem
✅ Details extracted - Issue: login, Severity: medium
🔄 Attempting to f

## 8. Key Features Demonstrated

### ✅ Dynamic Routing
- The agent adapts its workflow based on real-time analysis and conditions
- Multiple decision points create flexible execution paths

### ✅ Error Recovery  
- Automatic retry mechanisms with exponential backoff simulation
- Graceful fallback to escalation when automated recovery fails

### ✅ State Management
- Comprehensive state tracking across the entire workflow
- Persistent context maintained throughout the support process

### ✅ Multi-Tool Integration
- Seamless integration of mock external systems (logs, tickets, alerts)
- Realistic simulation of actual production tool interactions

### ✅ Intelligent Decision Making
- Context-aware routing based on severity, availability, and error conditions
- Adaptive behavior that evolves based on intermediate results

## 🎯 Production Readiness Considerations

1. **Replace Mock Tools**: Integrate with real logging, ticketing, and alerting systems
2. **Enhanced Error Handling**: Add circuit breakers and timeout mechanisms  
3. **Metrics & Monitoring**: Implement comprehensive observability
4. **Security**: Add authentication and authorization for tool access
5. **Scalability**: Design for concurrent execution and resource management

## 9. Conclusion

This PoC demonstrates the power and flexibility of LangGraph for building sophisticated support agents:

- **Adaptive Workflows**: Dynamic routing enables intelligent decision-making
- **Resilient Design**: Error recovery and retry mechanisms ensure reliability  
- **Extensible Architecture**: Easy to add new tools and decision points
- **State-Driven Logic**: Comprehensive state management enables complex workflows

The agent successfully handles various scenarios while maintaining robust error recovery and intelligent routing capabilities! 🚀